In [1]:
import pandas as pd
from datetime import datetime, timedelta

In [2]:
! git clone https://github.com/Klemersoncastro/google_trends_covid
%cd google_trends_covid/dados

Cloning into 'google_trends_covid'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 74 (delta 14), reused 32 (delta 11), pack-reused 37
Unpacking objects: 100% (74/74), done.
/content/google_trends_covid/dados


In [3]:
#Lendo base de dados
df_covid = pd.read_csv('casos_covid.csv', index_col=0)
df_trendDiario_br = pd.read_csv('sub-dados/1. trends/Trend diário - Brasil.csv')
df_trendDiario_am = pd.read_csv('sub-dados/1. trends/Trend diário - Amazonas.csv')
df_trendSemanal_br = pd.read_csv('sub-dados/1. trends/Trend semanal - Brasil.csv')
df_trendSemanal_am = pd.read_csv('sub-dados/1. trends/Trend semanal - Amazonas.csv')

In [4]:
# Base de dados do covid
df_covid

,date,estimated_population,estimated_population_2019,is_last,is_repeated,last_available_confirmed,last_available_confirmed_per_100k_inhabitants,last_available_date,last_available_death_rate,last_available_deaths,order_for_place,place_type,state,new_confirmed,new_deaths
1,2020-03-17,894470.0,881935.0,False,False,3,0.33539,2020-03-17,0.0000,0,1,state,AC,3,0
3,2020-03-18,894470.0,881935.0,False,False,3,0.33539,2020-03-18,0.0000,0,2,state,AC,0,0
5,2020-03-19,894470.0,881935.0,False,False,4,0.44719,2020-03-19,0.0000,0,3,state,AC,1,0
7,2020-03-20,894470.0,881935.0,False,False,7,0.78259,2020-03-20,0.0000,0,4,state,AC,3,0
9,2020-03-21,894470.0,881935.0,False,False,11,1.22978,2020-03-21,0.0000,0,5,state,AC,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1655203,2021-02-25,46289333.0,45919049.0,False,False,2014529,4352.03722,2021-02-25,0.0292,58873,367,state,SP,11889,345
1655850,2021-02-26,46289333.0,45919049.0,False,False,2026125,4377.08835,2021-02-26,0.0292,59129,368,state,SP,11596,256
1656497,2021-02-27,46289333.0,45919049.0,False,False,2037267,4401.15869,2021-02-27,0.0292,59428,369,state,SP,11142,299
1657144,2021-02-28,46289333.0,45919049.0,False,False,2041628,4410.57986,2021-02-28,0.0291,59493,370,state,SP,4361,65


In [5]:
#Selecionando apenas as colunas de interesse do covid
df_covid_brasil = df_covid[df_covid['place_type']=='state'][['date','new_confirmed','new_deaths']]
df_covid_amazonas = df_covid[(df_covid['place_type']=='state') & (df_covid['state']=='AM')][['date','new_confirmed','new_deaths']]

In [6]:
#Dataset com as colunas de interesse selecionadas
df_covid_brasil

,date,new_confirmed,new_deaths
1,2020-03-17,3,0
3,2020-03-18,0,0
5,2020-03-19,1,0
7,2020-03-20,3,0
9,2020-03-21,4,0
...,...,...,...
1655203,2021-02-25,11889,345
1655850,2021-02-26,11596,256
1656497,2021-02-27,11142,299
1657144,2021-02-28,4361,65


### Concatenando as trends com os casos de covid - Casos diários

In [7]:
# Agrupando os casos de covid por dia / casos diários
df_covidDiario_br = df_covid_brasil.groupby(by=['date']).sum().reset_index()
df_covidDiario_am = df_covid_amazonas.groupby(by=['date']).sum().reset_index()

In [8]:
# Casos diários de Covid no Brasil
df_covidDiario_br

,date,new_confirmed,new_deaths
0,2020-02-25,1,0
1,2020-02-26,0,0
2,2020-02-27,0,0
3,2020-02-28,1,0
4,2020-02-29,0,0
...,...,...,...
366,2021-02-25,68141,1576
367,2021-02-26,63530,1340
368,2021-02-27,58418,1252
369,2021-02-28,33288,772


In [9]:
# Trends diárias sobre o Covid
df_trendDiario_br

,date,covid,coronavirus,falta_de_ar,febre,tosse,perda_paladar,perda_olfato
0,2020-01-01,0,0,18,26,25,0,0
1,2020-01-02,0,0,23,32,30,0,0
2,2020-01-03,0,0,23,27,25,0,5
3,2020-01-04,0,0,21,39,29,0,5
4,2020-01-05,0,0,20,33,34,0,0
...,...,...,...,...,...,...,...,...
419,2021-02-23,83,56,59,88,75,17,26
420,2021-02-24,85,61,61,87,81,35,34
421,2021-02-25,88,69,51,94,75,41,38
422,2021-02-26,96,65,50,73,73,29,18


In [10]:
# Juntando as trends com os casos de covid no mesmo dataframe
df_casosDiario_br = df_trendDiario_br.merge(df_covidDiario_br, how='outer', on='date')
df_casosDiario_am = df_trendDiario_am.merge(df_covidDiario_am, how='outer', on='date')

In [11]:
#Selecionando os casos de covid até o dia 2021-02-27 (até onde foi coletado as trends)
df_casosDiario_br = df_casosDiario_br[df_casosDiario_br['date']<='2021-02-27']
df_casosDiario_am = df_casosDiario_am[df_casosDiario_am['date']<='2021-02-27']

In [12]:
df_casosDiario_br

,date,covid,coronavirus,falta_de_ar,febre,tosse,perda_paladar,perda_olfato,new_confirmed,new_deaths
0,2020-01-01,0.0,0.0,18.0,26.0,25.0,0.0,0.0,NaN,NaN
1,2020-01-02,0.0,0.0,23.0,32.0,30.0,0.0,0.0,NaN,NaN
2,2020-01-03,0.0,0.0,23.0,27.0,25.0,0.0,5.0,NaN,NaN
3,2020-01-04,0.0,0.0,21.0,39.0,29.0,0.0,5.0,NaN,NaN
4,2020-01-05,0.0,0.0,20.0,33.0,34.0,0.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
419,2021-02-23,83.0,56.0,59.0,88.0,75.0,17.0,26.0,63588.0,1358.0
420,2021-02-24,85.0,61.0,61.0,87.0,81.0,35.0,34.0,65387.0,1446.0
421,2021-02-25,88.0,69.0,51.0,94.0,75.0,41.0,38.0,68141.0,1576.0
422,2021-02-26,96.0,65.0,50.0,73.0,73.0,29.0,18.0,63530.0,1340.0


In [13]:
# Substituindo os valores nulos por zero
df_casosDiario_br = df_casosDiario_br.fillna(0)
df_casosDiario_am = df_casosDiario_am.fillna(0)

In [14]:
df_casosDiario_br

,date,covid,coronavirus,falta_de_ar,febre,tosse,perda_paladar,perda_olfato,new_confirmed,new_deaths
0,2020-01-01,0.0,0.0,18.0,26.0,25.0,0.0,0.0,0.0,0.0
1,2020-01-02,0.0,0.0,23.0,32.0,30.0,0.0,0.0,0.0,0.0
2,2020-01-03,0.0,0.0,23.0,27.0,25.0,0.0,5.0,0.0,0.0
3,2020-01-04,0.0,0.0,21.0,39.0,29.0,0.0,5.0,0.0,0.0
4,2020-01-05,0.0,0.0,20.0,33.0,34.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
419,2021-02-23,83.0,56.0,59.0,88.0,75.0,17.0,26.0,63588.0,1358.0
420,2021-02-24,85.0,61.0,61.0,87.0,81.0,35.0,34.0,65387.0,1446.0
421,2021-02-25,88.0,69.0,51.0,94.0,75.0,41.0,38.0,68141.0,1576.0
422,2021-02-26,96.0,65.0,50.0,73.0,73.0,29.0,18.0,63530.0,1340.0


In [15]:
# Mudando as colunas do tipo float para int
df_casosDiario_br[['covid','coronavirus','new_confirmed','new_deaths']] = df_casosDiario_br[['covid','coronavirus','new_confirmed','new_deaths']].astype(int)
df_casosDiario_am[['covid','coronavirus','new_confirmed','new_deaths']] = df_casosDiario_am[['covid','coronavirus','new_confirmed','new_deaths']].astype(int)

In [16]:
# Dataset de casos Diários do Brasil
df_casosDiario_br

,date,covid,coronavirus,falta_de_ar,febre,tosse,perda_paladar,perda_olfato,new_confirmed,new_deaths
0,2020-01-01,0,0,18.0,26.0,25.0,0.0,0.0,0,0
1,2020-01-02,0,0,23.0,32.0,30.0,0.0,0.0,0,0
2,2020-01-03,0,0,23.0,27.0,25.0,0.0,5.0,0,0
3,2020-01-04,0,0,21.0,39.0,29.0,0.0,5.0,0,0
4,2020-01-05,0,0,20.0,33.0,34.0,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...
419,2021-02-23,83,56,59.0,88.0,75.0,17.0,26.0,63588,1358
420,2021-02-24,85,61,61.0,87.0,81.0,35.0,34.0,65387,1446
421,2021-02-25,88,69,51.0,94.0,75.0,41.0,38.0,68141,1576
422,2021-02-26,96,65,50.0,73.0,73.0,29.0,18.0,63530,1340


In [17]:
# Dataset dos casos diários do Amazonas
df_casosDiario_am

,date,covid,coronavirus,falta_de_ar,febre,tosse,perda_paladar,perda_olfato,new_confirmed,new_deaths
0,2020-01-01,0,0,0.0,0.0,0.0,0.0,0.0,0,0
1,2020-01-02,0,0,0.0,15.0,0.0,0.0,0.0,0,0
2,2020-01-03,0,0,0.0,0.0,0.0,0.0,0.0,0,0
3,2020-01-04,0,0,0.0,33.0,16.0,0.0,0.0,0,0
4,2020-01-05,0,0,0.0,0.0,36.0,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...
419,2021-02-23,24,5,23.0,23.0,0.0,0.0,0.0,2147,51
420,2021-02-24,24,5,0.0,23.0,0.0,77.0,0.0,1608,69
421,2021-02-25,22,16,0.0,12.0,12.0,0.0,0.0,1420,86
422,2021-02-26,22,16,0.0,22.0,12.0,0.0,0.0,1572,56


In [18]:
df_casosDiario_br.to_csv('/content/Covid + Trends diário - Brasil.csv')
df_casosDiario_am.to_csv('/content/Covid + Trends diário - Amazonas.csv')

### Concatenando as trends com os casos de covid - Casos semanais

In [20]:
df_trendSemanal_br.iloc[[0,1,2,3,4,-5,-4,-3,-2,-1]]

,date,covid,coronavirus,falta_de_ar,febre,tosse,perda_paladar,perda_olfato
0,2020-01-05,0,0,26,39,32,1,1
1,2020-01-12,0,0,24,38,30,2,4
2,2020-01-19,0,3,25,48,24,3,0
3,2020-01-26,0,14,27,43,23,1,2
4,2020-02-02,0,5,25,41,26,0,1
55,2021-01-24,75,2,31,41,27,47,38
56,2021-01-31,68,2,33,38,24,36,34
57,2021-02-07,65,2,24,39,25,25,30
58,2021-02-14,75,3,32,42,28,33,29
59,2021-02-21,92,3,34,47,35,43,35


In [21]:
# Transformando coluna de datas para tipo date
df_covidDiario_br['date'] = pd.to_datetime(df_covidDiario_br['date'])
df_covidDiario_am['date'] = pd.to_datetime(df_covidDiario_am['date'])

In [22]:
'''
Função para agrupar os casos de covid por semana de acordo com as mesmas semanas das trends.
Parâmetros:
  df_casosDiarioCovid: dataframe de casos de covid diário com as colunas |date|new_confirmed|new_death
  df_trendSemanal: dataframe com as trends semanais.
Return:
  Retorna um dataframe já com as trends e casos de covid juntas. Por semana.
'''
def agruparCasosCovidPorSemana(df_casosDiarioCovid, df_trendSemanal) :
  list_dict_covid_casosPorSemana = []
  data_primeiro_caso_covid = df_casosDiarioCovid['date'].min()
  for data in df_trendSemanal['date'] :
    if pd.to_datetime(data) < pd.to_datetime(data_primeiro_caso_covid) :
      list_dict_covid_casosPorSemana.append({'date':data, 'new_confirmed':0, 'new_deaths':0})
    else :
      list_dict_covid_casosPorSemana.append(
          {'date': data, 
          'new_confirmed': df_casosDiarioCovid[(df_casosDiarioCovid['date']>=pd.to_datetime(data)) & (df_casosDiarioCovid['date']<=pd.to_datetime(data)+timedelta(days=6))]['new_confirmed'].sum(),
          'new_deaths': df_casosDiarioCovid[(df_casosDiarioCovid['date']>=pd.to_datetime(data)) & (df_casosDiarioCovid['date']<=pd.to_datetime(data)+timedelta(days=6))]['new_deaths'].sum()
          }
      )
  return pd.DataFrame(list_dict_covid_casosPorSemana)

In [23]:
# Agrupando os casos de covid por semana de acordo com as semanas das trends
df_casosSemanal_br = agruparCasosCovidPorSemana(df_covidDiario_br, df_trendSemanal_br)
df_casosSemanal_am = agruparCasosCovidPorSemana(df_covidDiario_am, df_trendSemanal_am)

In [25]:
df_casosSemanal_br.iloc[[0,1,2,3,-4,-3,-2,-1]]

,date,new_confirmed,new_deaths
0,2020-01-05,0,0
1,2020-01-12,0,0
2,2020-01-19,0,0
3,2020-01-26,0,0
56,2021-01-31,322189,7116
57,2021-02-07,314475,7573
58,2021-02-14,327047,7345
59,2021-02-21,380391,8290


In [26]:
df_casosSemanal_br = df_trendSemanal_br.merge(df_casosSemanal_br, how='inner', on='date')
df_casosSemanal_am = df_trendSemanal_am.merge(df_casosSemanal_am, how='inner', on='date')

In [27]:
df_casosSemanal_br.iloc[[0,1,2,3,-4,-3,-2,-1]]

,date,covid,coronavirus,falta_de_ar,febre,tosse,perda_paladar,perda_olfato,new_confirmed,new_deaths
0,2020-01-05,0,0,26,39,32,1,1,0,0
1,2020-01-12,0,0,24,38,30,2,4,0,0
2,2020-01-19,0,3,25,48,24,3,0,0,0
3,2020-01-26,0,14,27,43,23,1,2,0,0
56,2021-01-31,68,2,33,38,24,36,34,322189,7116
57,2021-02-07,65,2,24,39,25,25,30,314475,7573
58,2021-02-14,75,3,32,42,28,33,29,327047,7345
59,2021-02-21,92,3,34,47,35,43,35,380391,8290


In [28]:
df_casosSemanal_am.iloc[[0,1,2,3,-4,-3,-2,-1]]

,date,covid,coronavirus,falta_de_ar,febre,tosse,perda_paladar,perda_olfato,new_confirmed,new_deaths
0,2020-01-05,0,0,15,45,34,0,0,0,0
1,2020-01-12,0,0,23,20,44,0,0,0,0
2,2020-01-19,0,5,5,40,9,0,0,0,0
3,2020-01-26,0,20,10,9,46,0,0,0,0
56,2021-01-31,56,4,41,60,19,0,10,15760,916
57,2021-02-07,47,4,16,36,46,0,0,12568,885
58,2021-02-14,36,3,13,40,20,10,11,10928,590
59,2021-02-21,27,2,24,28,34,9,10,10349,422


In [29]:
df_casosSemanal_br.to_csv('/content/Covid + Trends semanais - Brasil.csv')
df_casosSemanal_am.to_csv('/content/Covid + Trends semanais - Amazonas.csv')